In [1]:
import math
import numpy as np
from numba import jit, njit, prange
from joblib import Parallel, delayed
from functools import partial
import multiprocessing as mp
from multiprocessing import Pool, Value, Array
from ipyparallel import Client
import pandas as pd
import timeit
%load_ext cython

**1**. (100 points)

Write a predicate function `is_prime` that efficiently checks whether a number is prime. Use this to write a second function `primes_between` that returns the prime numbers between two integers as a `numpy` array.

- (10 points) Do this in regular Python 
- (10 points) Accelerate using `numba` (serial version) 
- (15 points) Accelerate using `numba` (parallel version)
- (10 points) Accelerate using `cython` (serial version) 
- (15 points) Accelerate using `cython` (parallel version)
- (10 points) Report the speed-up multiplier as an integer of the `numba` and `cython` serial and parallel versions using `timeit` in a DataFrame for the numbers between 0 and 1,000,000
- (10 points each) Run the serial version of the python `primes_between` function in parallel using
    - `multiprocessing`
    - `joblib`
    - `ipyparallel`

- (10 points) Do this in regular Python 



*Solution from Midterm 1 is used for is_prime( ) function*

In [2]:
def is_prime(n):
    """Returns True if a given integer n is prime and false otherwise"""

    if n == 2:
        return True
    elif n < 2 or n % 2 == 0:
        return False
    else:
        for i in range(3, int(np.sqrt(n))+1, 2):
            if n % i == 0:
                return False
    return True

In [3]:
def primes_between(n1, n2):
    """Returns prime numbers between n1 and n2 (exclusive on both ends) as a numpy array"""
    
    # Check to see which argument is larger
    if n1 > n2:
        n1, n2 = n2, n1
    
    # Initialize output and loop through all numbers between n1 and n2
    primes = []
    for num in range(n1 + 1, n2):
        if is_prime(num):
            primes.append(num)
    
    # Return result
    return np.array(primes)

In [4]:
# Prove that function works
primes_between(0, 50)

array([ 2,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47])

In [5]:
%%timeit
# Time function
primes_between(0, 1000)

1.29 ms ± 9.83 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


- (10 points) Accelerate using `numba` (serial version) 



In [5]:
@jit(nopython=True, cache=True)
def is_prime_numba_serial(n):
    """Returns True if a given integer n is prime and false otherwise"""

    if n == 2:
        return True
    elif n < 2 or n % 2 == 0:
        return False
    else:
        for i in range(3, int(np.sqrt(n))+1, 2):
            if n % i == 0:
                return False
    return True

In [6]:
@jit(nopython=True, cache=True)
def primes_between_numba_serial(n1, n2):
    """Returns prime numbers between n1 and n2 (exclusive on both ends) as a numpy array"""
    
    # Check to see which argument is larger
    if n1 > n2:
        n1, n2 = n2, n1
    
    # Initialize output and loop through all numbers between n1 and n2
    primes = []
    for num in range(n1 + 1, n2):
        if is_prime_numba_serial(num):
            primes.append(num)
    
    # Return result
    return np.array(primes)

In [7]:
# Prove that function works
primes_between_numba_serial(0, 50)

array([ 2,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47])

In [12]:
%%timeit
# Time function
primes_between_numba_serial(0, 1000)

21.9 µs ± 273 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


- (15 points) Accelerate using `numba` (parallel version)



**Need to fix this problem**

In [12]:
@njit(parallel=True)
def is_prime_numba_parallel(n):
    """Returns True if a given integer n is prime and false otherwise"""
    
    if n == 2:
        return True
    elif n < 2 or n % 2 == 0:
        return False
    else:
        for i in prange(3, int(np.sqrt(n))+1, 2):
            if n % i == 0:
                return False
    return True

In [10]:
is_prime_numba_parallel.parallel_diagnostics(level=4)

 
 Parallel Accelerator Optimizing:  Function is_prime_numba_parallel, <ipython-
input-8-43e8d3039c1b> (1)  


Parallel loop listing for  Function is_prime_numba_parallel, <ipython-input-8-43e8d3039c1b> (1) 
----------------------------------------------------------------------------|loop #ID
@njit(parallel=True)                                                        | 
def is_prime_numba_parallel(n):                                             | 
    """Returns True if a given integer n is prime and false otherwise"""    | 
                                                                            | 
    if n == 2:                                                              | 
        return True                                                         | 
    elif n < 2 or n % 2 == 0:                                               | 
        return False                                                        | 
    else:                                                                 

**May want to try using numpy array with specified data type. However, a new array will be created every time, which is definitely sub-optimal**

In [ ]:
@njit(parallel=True)
def primes_between_numba_parallel(n1, n2):
    """Returns prime numbers between n1 and n2 (exclusive on both ends) as a numpy array"""
    
    # Check to see which argument is larger
    if n1 > n2:
        n1, n2 = n2, n1
    
    # Initialize output and loop through all numbers between n1 and n2
    primes = [np.complex64(i) for i in range(0)]
    for num in prange(n1 + 1, n2):
        if is_prime_numba_parallel(num):
            #primes.append(num)
            primes += [num]
    
    
    # Return result
    return np.array(primes)

In [29]:
@njit(parallel=True)
def primes_between_numba_parallel(n1, n2):
    """Returns prime numbers between n1 and n2 (exclusive on both ends) as a numpy array"""
    
    # Check to see which argument is larger
    if n1 > n2:
        n1, n2 = n2, n1
    
    # Initialize output and loop through all numbers between n1 and n2
    primes = []
    for num in prange(n1 + 1, n2):
        if is_prime_numba_parallel(num):
            primes.append(num)
    
    # Return result
    return np.array(primes)

In [30]:
# Prove that function works
primes_between_numba_parallel(0, 10)

LoweringError: Failed in nopython mode pipeline (step: nopython mode backend)
Cannot lower constant of type 'list(complex64)'

File "<ipython-input-29-65af957d9762>", line 11:
def primes_between_numba_parallel(n1, n2):
    <source elided>
    primes = [np.complex64(i) for i in range(0)]
    for num in prange(n1 + 1, n2):
    ^

[1] During: lowering "id=3[LoopNest(index_variable = parfor_index.319, range = ($40.4, n2, 1))]{81: <ir.Block at <ipython-input-29-65af957d9762> (11)>, 244: <ir.Block at <ipython-input-29-65af957d9762> (14)>, 246: <ir.Block at <ipython-input-29-65af957d9762> (11)>}Var(parfor_index.319, <ipython-input-29-65af957d9762>:11)" at <ipython-input-29-65af957d9762> (11)

In [1]:
primes_between_numba_parallel.parallel_diagnostics(level=4)

NameError: name 'primes_between_numba_parallel' is not defined

- (10 points) Accelerate using `cython` (serial version) 



In [8]:
%%cython -a

import cython
import numpy as np

def is_prime_cython_serial(int n):
    """Returns True if a given integer n is prime and false otherwise"""
    
    cdef int i
    if n == 2:
        return True
    elif n < 2 or n % 2 == 0:
        return False
    else:
        for i in range(3, int(np.sqrt(n)) + 1, 2):
            if n % i == 0:
                return False
    return True

def primes_between_cython_serial(int n1, int n2, list primes):
    """Returns prime numbers between n1 and n2 (exclusive on both ends) as a numpy array"""
    
    # Check to see which argument is larger
    if n1 > n2:
        n1, n2 = n2, n1
    
    # Loop through all numbers between n1 and n2
    cdef int num
    for num in range(n1 + 1, n2):
        if is_prime_cython_serial(num):
            primes.append(num)
    
    # Return result
    return np.array(primes)

In [10]:
# Prove that function works
primes_between_cython_serial(1, 50, [])

array([ 2,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47])

In [11]:
%%timeit
# Time function
primes_between_cython_serial(1, 1000,[])

733 µs ± 4.93 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


- (15 points) Accelerate using `cython` (parallel version)



**Need to fix this problem**

In [19]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --force -I /usr/local/opt/libomp/include -L /usr/local/opt/libomp/lib

import cython
from cython.parallel import parallel, prange
import math
import numpy as np

def is_prime_cython_parallel(int n, bool):
    """Returns True if a given integer n is prime and false otherwise"""
    
    cdef int i
    with cython.nogil, parallel(): 
        if n == 2:
            return True
        elif n < 2 or n % 2 == 0:
            return False
        else:
            for i in prange(3, int(np.sqrt(n))+1, 2):
                if n % i == 0:
                    return False
        return True

def primes_between_cython_parallel(int n1, int n2, list primes):
    """Returns prime numbers between n1 and n2 (exclusive on both ends) as a numpy array"""
    
    # Loop through all numbers between n1 and n2
    cdef int num
    with cython.nogil, parallel():    
        for num in prange(n1 + 1, n2):
            if is_prime_cython_parallel(num):
                primes.append(num)
    
    # Return result
    return np.array(primes)


Error compiling Cython file:
------------------------------------------------------------
...
        if n == 2:
            return True
        elif n < 2 or n % 2 == 0:
            return False
        else:
            for i in prange(3, int(sqrt(n))+1, 2):
                                  ^
------------------------------------------------------------

/home/jovyan/.cache/ipython/cython/_cython_magic_bf499edb7401d0e3f1a02f4abaca8323.pyx:17:35: undeclared name not builtin: sqrt

Error compiling Cython file:
------------------------------------------------------------
...
        if n == 2:
            return True
        elif n < 2 or n % 2 == 0:
            return False
        else:
            for i in prange(3, int(sqrt(n))+1, 2):
                                          ^
------------------------------------------------------------

/home/jovyan/.cache/ipython/cython/_cython_magic_bf499edb7401d0e3f1a02f4abaca8323.pyx:17:43: stop argument must be numeric

Error compiling Cytho

- (10 points) Report the speed-up multiplier as an integer of the `numba` and `cython` serial and parallel versions using `timeit` in a DataFrame for the numbers between 0 and 1,000,000



*Values in data frame are calculated as (time for serial version in python) / (time for sped up version). A larger integer value denotes a greater speed-up*

In [70]:
# Create dataframe
df = pd.DataFrame(dict(Serial = [0, 0], Parallel = [0, 0]), index = ['numba', 'cython'])

# Set upper bound
upper = 1000000

# Calculate reference time
ref = %timeit -o -r3 -n3 primes_between(0, upper)
ref = ref.average

# numba serial
numba_serial = %timeit -o -r3 -n3 primes_between_numba_serial(0, upper)
numba_serial = numba_serial.average
df.iloc[0, 0] = int(np.round(ref / numba_serial))

# numba parallel
#numba_parallel = %timeit -o -r3 -n3 primes_between_numba_parallel(0, upper)
#numba_parallel = numba_parallel.average
#df.iloc[0, 1] = int(np.round(ref / numba_parallel))

# cython serial
cython_serial = %timeit -o -r3 -n3 primes_between_cython_serial(0, upper, [])
cython_serial = cython_serial.average
df.iloc[1, 0] = int(np.round(ref / cython_serial))

# cython parallel
#cython_parallel = %timeit -o -r3 -n3 primes_between_cython_parallel(0, upper)
#cython_parallel = cython_parallel.average
#df.iloc[1, 1] = int(np.round(ref / cython_parallel))

df

14.8 ms ± 338 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)
437 µs ± 18.1 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)
7.67 ms ± 55.1 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


,Serial,Parallel
numba,34,0
cython,2,0


- (10 points) `multiprocessing`



In [23]:
# Create array with appropriate partitions
loop_array = np.array([0, 125000, 125001, 250000, 250001, 375000, 375001, 500000, 
                       500001, 625000, 625001, 750000, 750001, 875000, 875001, 1000000])

# Parallelize using multiprocessing
with mp.Pool(processes = 8) as pool:
    res = pool.starmap(primes_between, np.array_split(loop_array, 8))
    
# Show result
np.concatenate(res)

array([     2,      3,      5, ..., 999961, 999979, 999983])

- (10 points) `joblib`



In [8]:
res = Parallel(n_jobs=8)(delayed(primes_between)(max(0, i - 124999), i) for i in range(0, 1000001, 125000))
np.concatenate(res).astype('int')

array([     2,      3,      5, ..., 999961, 999979, 999983])

- (10 points) `ipyparallel`

In [26]:
# Connect to cluster of remote engines
rc = Client()
dv = rc[:]

In [27]:
# Define nested primes_between function
def primes_between_ipyparallel(n1, n2):
    """Returns prime numbers between n1 and n2 (exclusive on both ends) as a numpy array"""
    
    # Import numpy
    import numpy as np
    
    # Define predicate function with "primes_between" so that it is recognized within cluser
    def is_prime_ipyparallel(n):
        """Returns True if a given integer n is prime and false otherwise"""

        # Function
        if n == 2:
            return True
        elif n < 2 or n % 2 == 0:
            return False
        else:
            for i in range(3, int(np.sqrt(n))+1, 2):
                if n % i == 0:
                    return False
        return True
    
    # Check to see which argument is larger
    if n1 > n2:
        n1, n2 = n2, n1
    
    # Initialize output and loop through all numbers between n1 and n2
    primes = []
    for num in range(n1 + 1, n2):
        if is_prime_ipyparallel(num):
            primes.append(num)
    
    # Return result
    return np.array(primes)

In [30]:
res = dv.map_sync(primes_between_ipyparallel, [0, 250001, 500001, 750001], [250000, 500000, 750000, 1000000])
np.concatenate(res)

array([     2,      3,      5, ..., 999961, 999979, 999983])